In [1]:
import os
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers as ly

In [2]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]

In [3]:
tf.__version__

'2.3.0'

In [4]:
tf.test.is_built_with_cuda()

True

In [5]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 37s 0us/step


In [6]:
X_train.shape

(50000, 32, 32, 3)

In [7]:
y_train.shape

(50000, 1)

In [10]:
classes = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck"
]

In [11]:
classes[y_train[3][0]]

'deer'

In [13]:
y_train[:3]

array([[6],
       [9],
       [9]], dtype=uint8)

### Scale images

In [15]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [17]:
y_train_categorical = keras.utils.to_categorical(
    y_train, num_classes=len(classes), dtype="float32"
)

y_test_categorical = keras.utils.to_categorical(
    y_test, num_classes=len(classes), dtype="float32"
)

In [19]:
y_test_categorical[:5]

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]], dtype=float32)

### Model building and training

In [21]:
def get_model():
    model = Sequential([
        ly.Flatten(input_shape=(32, 32, 3)),
        ly.Dense(3000, activation="relu"),
        ly.Dense(1000, activation="relu"),
        ly.Dense(10, activation="sigmoid"),
    ])
    
    model.compile(optimizer="SGD",
                 loss="categorical_crossentropy",
                 metrics=["accuracy"])
    return model

In [27]:
train_tf_dataset = tf.data.Dataset.from_tensor_slices(
    (X_train_scaled, y_train_categorical))
test_tf_dataset = tf.data.Dataset.from_tensor_slices(
    (X_test_scaled, y_test_categorical))

In [28]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [29]:
strategy.num_replicas_in_sync

1

In [30]:
BATCH_SIZE_PER_REPLICA = 250
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

In [32]:
train_dataset = train_tf_dataset.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
test_dataset = test_tf_dataset.batch(BATCH_SIZE)

In [36]:
%%timeit -n1 -r1 
with strategy.scope():
    gpu_model = get_model()
    gpu_model.fit(train_dataset, epochs=50)

Epoch 1/50
200/200 [==============================] - 4s 20ms/step - loss: 2.0498 - accuracy: 0.2759
Epoch 2/50
200/200 [==============================] - 4s 18ms/step - loss: 1.8568 - accuracy: 0.3446
Epoch 3/50
200/200 [==============================] - 4s 18ms/step - loss: 1.7933 - accuracy: 0.3691 0s - loss: 1.7952 
Epoch 4/50
200/200 [==============================] - 4s 18ms/step - loss: 1.7497 - accuracy: 0.3866
Epoch 5/50
200/200 [==============================] - 3s 17ms/step - loss: 1.7148 - accuracy: 0.3994
Epoch 6/50
200/200 [==============================] - 4s 18ms/step - loss: 1.6854 - accuracy: 0.4111
Epoch 7/50
200/200 [==============================] - 4s 18ms/step - loss: 1.6596 - accuracy: 0.4211
Epoch 8/50
200/200 [==============================] - 4s 18ms/step - loss: 1.6365 - accuracy: 0.4306
Epoch 9/50
200/200 [==============================] - 4s 18ms/step - loss: 1.6156 - accuracy: 0.4369
Epoch 10/50
200/200 [==============================] - 4s 18ms/step - lo

In [37]:
%%timeit -n1 -r1 
with tf.device("/CPU"):
    cpu_model = get_model()
    cpu_model.fit(train_dataset, epochs=50)

Epoch 1/50
200/200 [==============================] - 35s 175ms/step - loss: 2.0424 - accuracy: 0.2734
Epoch 2/50
200/200 [==============================] - 24s 122ms/step - loss: 1.8543 - accuracy: 0.3437
Epoch 3/50
200/200 [==============================] - 24s 122ms/step - loss: 1.7886 - accuracy: 0.3698
Epoch 4/50
200/200 [==============================] - 23s 117ms/step - loss: 1.7432 - accuracy: 0.3875
Epoch 5/50
200/200 [==============================] - 24s 118ms/step - loss: 1.7070 - accuracy: 0.4026
Epoch 6/50
200/200 [==============================] - 23s 117ms/step - loss: 1.6764 - accuracy: 0.4151
Epoch 7/50
200/200 [==============================] - 23s 117ms/step - loss: 1.6500 - accuracy: 0.4240
Epoch 8/50
200/200 [==============================] - 24s 118ms/step - loss: 1.6266 - accuracy: 0.4317
Epoch 9/50
200/200 [==============================] - 23s 117ms/step - loss: 1.6056 - accuracy: 0.4391
Epoch 10/50
200/200 [==============================] - 23s 117ms/step - l